In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import sys

In [ ]:
def get_links(result_link):
    """Gets all article links on a given page on a news site"""
    links=[]
    r = requests.get(result_link)
    soup = BeautifulSoup(r.text, 'html.parser')
    links_results = soup.findAll('h2', {"class":"entry-title"}) #Gets article link wrappers
    
    for result in links_results:
        links.append(result.find('a').get('href'))

    return links

def get_article(article_link):
    """Takes in a link to an article and returns the formatted article among other info"""
    title = ""
    article = ""
    author = ""
    date = ""
    
    r = requests.get(article_link)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    #Gets the article and other relevant info
    title = soup.find('h1', class_='title entry-title').text
    author = soup.find('a', {'rel': 'author' }).text
    date = soup.find('abbr', class_='date').text    
    article_body = soup.find(class_='entry')    

    for p in article_body.findAll('p'):
        excerpt = p.text.replace(u'\xa0', u' ')
        article = article + " " + excerpt
        
    return title, author, date, article

In [ ]:
titles = []
authors = []
dates = []
articles = []
MAX_PAGES = 55

for i in range(1, MAX_PAGES):    
    links = get_links('http://www.globalclimatescam.com/page/{}/'.format(i))
    
    for link in links:
        title, author, date, article = get_article(link)
        
        titles.append(title)    
        authors.append(author)
        dates.append(date)
        articles.append(article)
        
    sys.stdout.write("\rPage {} done, total articles: {}".format(i, len(articles)))
    sys.stdout.flush()

In [ ]:
df = pd.DataFrame(data={'title': titles, 'author': authors, 'date': dates, 'article': articles}).drop_duplicates(keep='first')
df['denial?'] = [1]*len(df.index)
print('Num. articles: {}'.format(len(df)))
df.head(5)

In [ ]:
df.to_csv('../../data/right/data_global_climate_scam.csv', index=False)